In [ ]:
import os
from google.colab import drive
from os.path import join as jn
import shutil

# 구글 드라이브 접근
ROOT = "/content/drive"
try:
  drive.mount(ROOT, force_remount=True)
except:
  drive.mount(ROOT)

# 본인 경로에 맞게 수정하면 됨.
PATH = jn(ROOT, "MyDrive/Task/plastic-segmentation/Data")
# PATH = jn(ROOT, "MyDrive/Task/plastic-segmentation/Sample_data")

# 구글 드라이브 경로에서 '/content/sample_data/data"로 복사 시
# shutil.copytree(PATH +"/train", "./sample_data/data/train")
# shutil.copytree(PATH +"/test", "./sample_data/data/test")

Mounted at /content/drive


# init

In [ ]:
import os
import json
from tqdm import tqdm
from glob import glob

import numpy as np
# import pandas as pd
import cv2 as cv
from PIL import Image

import matplotlib.pyplot as plt
import matplotlib.patches as patches
from pycocotools.coco import COCO

import torch
import torch.optim as optim
from torch  import nn, Tensor

BASE = os.getcwd()
PATH = PATH # gdrive
# PATH = f"{BASE}/sample_data/data"   # colab
# PATH = f"{BASE}/assets/data"        # github

In [ ]:
# version 1 : @qkrwjdduf159

## train
trn_path = f"{PATH}/train/annotation"
file_list = os.listdir(trn_path)

train_files = []
for file in tqdm(file_list):
    dir = trn_path + '/' + file
    json_list = glob(dir + '/' + '*.json')
    train_files.append(json_list)

train_json_list = []
for files in tqdm(train_files):
    for json_file in tqdm(files):
        train_json_list.append(json_file)

## test
tst_PATH = f"{PATH}/test/annotations"
file_list = os.listdir(tst_PATH)

test_files = []
for file in tqdm(file_list):
    dir = tst_PATH + '/' + file
    json_list = glob(dir + '/' + '*.json')
    test_files.append(json_list)

test_json_list = []
for files in tqdm(test_files):
    for json_file in tqdm(files):
        test_json_list.append(json_file)

100%|██████████| 1000/1000 [00:00<00:00, 1985939.39it/s]

100%|██████████| 1000/1000 [00:00<00:00, 1808669.25it/s]

100%|██████████| 1000/1000 [00:00<00:00, 1076290.48it/s]

100%|██████████| 100/100 [00:00<00:00, 594094.05it/s]

100%|██████████| 100/100 [00:00<00:00, 884874.26it/s]

100%|██████████| 100/100 [00:00<00:00, 651289.44it/s]

100%|██████████| 4/4 [00:00<00:00, 189.45it/s]


In [ ]:
# version 2 @AshbeeKim
trn = os.path.join(PATH, "train")
tst = os.path.join(PATH, "test")

baseDF = {"kind" : [], "label" : [], "metainfo_id" : [], "feature" : [], "image_path" : [], "annot_path": []}
trn_num, tst_num = 0, 0
for fpath in tqdm([trn, tst]):
    kind = os.path.basename(fpath)
    BDIR = sorted(os.listdir(fpath))    # image, annotation(s)
    for bdir in BDIR:
        for dirs in sorted(os.listdir(jn(fpath, bdir))):
            dpath = jn(fpath, bdir, dirs)
            if bdir.lower()=='image':
                paths = sorted(glob(dpath + "/*.jpg"))
                baseDF['image_path'].extend(paths)
                kinds = [kind for cnt in range(len(paths))]
                baseDF['kind'].extend(kinds)
                if kind=='train' : trn_num += len(paths)
                elif kind=='test' : tst_num += len(paths)
                fnames = [os.path.basename(fname) for fname in paths]
                labels = list(map(lambda x: (x.split('_')[0]), fnames))
                baseDF['label'].extend(labels)
                metaIds = list(map(lambda x: int(x.split('_')[1]), fnames))
                baseDF['metainfo_id'].extend(metaIds)
                feats = list(map(lambda x: int(x.split('_')[-1][:-4]), fnames))
                baseDF['feature'].extend(feats)
            else:
                paths = sorted(glob(dpath + "/*.json"))
                baseDF['annot_path'].extend(paths)

100%|██████████| 2/2 [00:17<00:00,  8.51s/it]


# 수정 point_1

In [ ]:
# cotributed by @qkrwjdduf156, @AshbeeKim
import os
import numpy as np
import torch
from PIL import Image

# 추가 수정 중
# from torchvision import datasets
# torch.utils.data.Dataset
class CustomDataset:
    def __init__(self, base_info, transforms = None, mode = 'train', train_size = None, test_size = None):
        self.mode = mode.lower()
        self.infos = base_info
        self.transforms = transforms

        if self.mode == 'train':
            self.images = (self.infos['image_path'])[:train_size]
            self.annots = (self.infos['annot_path'])[:train_size]

        elif self.mode == 'test':
            self.images = (self.infos['image_path'])[train_size:train_size+test_size]
            self.annots = (self.infos['annot_path'])[train_size:train_size+test_size]

    def __getitem__(self, idx):
        json_path = self.annots[idx]
        image_path = self.images[idx]

        if self.mode == 'train':
            image = Image.open(image_path).convert('RGB')   # (C, H, W)
            mask = np.zeros(image.copy())   # (C, H, W) # BK
            
            with open(json_path, 'r') as f:
                annot = json.loads(f.read())

            boxes = []
            segmentations = []
            labels = []
            for i in range(len(annot['annotations'])):

                segmentation = annot['annotations'][i]['segmentation'][0]
                bbox = annot['annotations'][i]['bbox']
                label = annot['annotations'][i]['category_id']
                xmin, ymin, width, height = bbox[0],bbox[1],bbox[2],bbox[3]
                xmin, ymin, xmax, ymax = xmin, ymin, xmin + width, ymin + height
                
                boxes.append([xmin, ymin, xmax, ymax])
                segmentations.append([segmentation])
                labels.append(label)

            target = {}
            target['boxes'] = torch.as_tensor(boxes, dtype = torch.float32)
            target['labels'] = torch.ones((labels, ), dtype = torch.int64)
            target['segmentation'] = segmentations
            target['image_id'] = image_path.split('/')[-1]

            if self.transforms is not None:
                image, target = self.transforms(image, target)

            return image, target

        elif self.model == 'test':
            image = Image.open(image_path).convert('RGB')
            with open(json_path, 'r') as f:
                annot = json.loads(f.read())
            
            target = {}
            target['image_id'] = image_path.split('/')[-1]
            return image, target
        
    def __len__(self):
        return len(self.images)

* fastrcnn

In [ ]:
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained = True)

num_classes = 5 # 4 class + background
in_features = model.roi_heads.box_predictor.cls_score.in_features

model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

Downloading: "https://download.pytorch.org/models/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth" to /root/.cache/torch/hub/checkpoints/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth


  0%|          | 0.00/160M [00:00<?, ?B/s]

In [ ]:
model.eval()

FasterRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=0.0)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=0.0)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=0.0)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256, eps=0.0)
          (relu): ReLU(

In [ ]:
from torchvision import transforms as T

train_dataset = CustomDataset(baseDF, mode = 'train', train_size=trn_num, test_size=tst_num)
input_image = train_dataset.__getitem__(23)[0]
preprocess = T.Compose([
                        T.Resize(256),
                        T.ToTensor(),
                        T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
input_tensor = preprocess(input_image)
input_batch = input_tensor.unsqueeze(0) # torch.Size([1, 3, 2048, 2048]) >>> (resize to 256*256) >>> torch.Size([1, 3, 256, 256])

In [ ]:
if torch.cuda.is_available():
    input_batch = input_batch.to('cuda')
    model.to('cuda')

# with torch.no_grad():
#     output = model(input_batch)[0]['out'][0]
# output_predictions = output.argmax(0)

In [ ]:
model(input_batch)[0]   # dict_keys(['boxes', 'labels', 'scores'])
model(input_batch)[0]['scores']

tensor([0.5513, 0.4587, 0.4472, 0.4444, 0.4397, 0.4277, 0.4276, 0.4151, 0.4149,
        0.4126, 0.4120, 0.4078, 0.4017, 0.4017, 0.4007, 0.3998, 0.3976, 0.3965,
        0.3963, 0.3960, 0.3958, 0.3923, 0.3875, 0.3839, 0.3839, 0.3820, 0.3797,
        0.3789, 0.3783, 0.3759, 0.3736, 0.3726, 0.3724, 0.3722, 0.3718, 0.3717,
        0.3709, 0.3704, 0.3673, 0.3659, 0.3648, 0.3642, 0.3631, 0.3614, 0.3596,
        0.3594, 0.3581, 0.3549, 0.3534, 0.3532, 0.3499, 0.3498, 0.3497, 0.3480,
        0.3475, 0.3445, 0.3434, 0.3428, 0.3425, 0.3421, 0.3411, 0.3409, 0.3401,
        0.3392, 0.3376, 0.3362, 0.3360, 0.3353, 0.3339, 0.3338, 0.3326, 0.3325,
        0.3325, 0.3324, 0.3323, 0.3312, 0.3307, 0.3303, 0.3299, 0.3284, 0.3274,
        0.3271, 0.3267, 0.3264, 0.3256, 0.3245, 0.3236, 0.3232, 0.3231, 0.3224,
        0.3209, 0.3207, 0.3199, 0.3197, 0.3191, 0.3191, 0.3190, 0.3186, 0.3184,
        0.3183], device='cuda:0', grad_fn=<IndexBackward0>)

* unet

In [ ]:
import torch
model = torch.hub.load('mateuszbuda/brain-segmentation-pytorch', 'unet',
    in_channels=3, out_channels=1, init_features=32, pretrained=True)


from torchvision import transforms as T

train_dataset = CustomDataset(baseDF, mode = 'train', train_size=trn_num, test_size=tst_num)
input_image = train_dataset.__getitem__(23)
m, s = np.mean(input_image[0], axis=(0, 1)), np.std(input_image[0], axis=(0, 1))
preprocess = T.Compose([
                        # T.Resize(256),
                        T.ToTensor(),
                        T.Normalize(mean=m, std=s),
])
input_tensor = preprocess(input_image[0])
input_batch = input_tensor.unsqueeze(0) # torch.Size([1, 3, 2048, 2048]) >>> (resize to 256*256) >>> torch.Size([1, 3, 256, 256])

Using cache found in /root/.cache/torch/hub/mateuszbuda_brain-segmentation-pytorch_master


In [ ]:
if torch.cuda.is_available():
    input_batch = input_batch.to('cuda')
    model = model.to('cuda')

with torch.no_grad():
    output = model(input_batch)

In [ ]:
print(torch.round(output))

tensor([[[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]]], device='cuda:0')


In [ ]:
model.eval()

UNet(
  (encoder1): Sequential(
    (enc1conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (enc1norm1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (enc1relu1): ReLU(inplace=True)
    (enc1conv2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (enc1norm2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (enc1relu2): ReLU(inplace=True)
  )
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (encoder2): Sequential(
    (enc2conv1): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (enc2norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (enc2relu1): ReLU(inplace=True)
    (enc2conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (enc2norm2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, tra

* fasterrcnn

In [59]:
import torchvision
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator

backbone = torchvision.models.mobilenet_v2(pretrained=True).features
backbone.out_channels = 1280

anchor_generator = AnchorGenerator(sizes=((32, 64, 128, 256, 512),),
                                   aspect_ratios=((0.5, 1.0, 2.0),))

roi_pooler = torchvision.ops.MultiScaleRoIAlign(featmap_names=['0'],
                                                output_size=7,
                                                sampling_ratio=2)

model = FasterRCNN(backbone,
                   num_classes=5,
                   rpn_anchor_generator=anchor_generator,
                   box_roi_pool=roi_pooler)

In [60]:
from torchvision import transforms as T

train_dataset = CustomDataset(baseDF, mode = 'train', train_size=trn_num, test_size=tst_num)
input_image = train_dataset.__getitem__(23)[0]
m, s = np.mean(input_image, axis=(0, 1)), np.std(input_image, axis=(0, 1))
preprocess = T.Compose([
                        # T.Resize(256),
                        T.ToTensor(),
                        T.Normalize(mean=m, std=s),
])
input_tensor = preprocess(input_image)
input_batch = input_tensor.unsqueeze(0) # torch.Size([1, 3, 2048, 2048]) >>> (resize to 256*256) >>> torch.Size([1, 3, 256, 256])

*** 현재 수정 중 ***

In [62]:
# if torch.cuda.is_available():
#     input_batch = input_batch.to('cuda')
#     model = model.to('cuda')

# with torch.no_grad():
#     output = model(input_batch)

* maskrcnn

In [ ]:
# %%writefile ./CaliberDetector.py    # customize 끝내고 run

# detection
from collection import OrderedDict
from torch import nn
from torchvision.ops import MultiScaleRoIAlign
from torchvision._internally_replaced_utils import load_state_dict_from_url
from torchvision.models.detection._utils import overwrite_eps
from torchvision.models.detection.faster_rcnn import FasterRCNN
from torchvision.models.detection.backbone_utils import resnet_fpn_backbone, _validate_trainable_layers



In [ ]:
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor
# MaskRCNN
# MaskRCNNHeads

def get_model_instance_segmentation(num_classes):
    model = torchvision.models.detection.maskrcnn_resnet50_fpn(pretrained=True)

    in_features = model.roi_heads.box_predictor.cls_score.in_features
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

    in_features_mask = model.roi_heads.mask_predictor.conv5_mask.in_channels
    hidden_layer = 256

    model.roi_heads.mask_predictor = MaskRCNNPredictor(in_features_mask,
                                                       hidden_layer,
                                                       num_classes)

    return model

In [ ]:
!pip install -U torchvision

In [ ]:
from torchvision import utils
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
dataset = CustomDataset(baseDF, mode = 'train', train_size=trn_num, test_size=tst_num)
data_loader = torch.utils.data.DataLoader(
 dataset, batch_size=5, shuffle=True, num_workers=4,
 collate_fn=utils.data._utils.collate)
# For Training
images,targets = next(iter(data_loader))
images = list(image for image in images)
targets = [{k: v for k, v in t.items()} for t in targets]
output = model(images,targets)   # Returns losses and detections
# For inference
model.eval()
x = [torch.rand(3, 300, 400), torch.rand(3, 500, 400)]
predictions = model(x)           # Returns predictions

In [ ]:
from torchvision iumport datasets

train_dataset = datasets.ImageFolder(root=PATH+'/train', trainsform = train_trainsform)

In [ ]:
from torchvision import transforms as T

def get_transform(train):
    transforms = []
    transforms.append(T.ToTensor())
    if train:
        transforms.append(T.RandomHorizontalFlip(0.5))
    return T.Compose(transforms)


transform = T.Compose([
        T.Resize(256),
        T.RandomCrop(224),
        T.RandomHorizontalFlip(),
        T.ToTensor(),
        T.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]) 

train_transform = T.Compose([
        T.Resize(256),
        T.RandomCrop(224),
        T.RandomHorizontalFlip(),
        T.ToTensor(),
        T.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]) 

transform = T.Compose([
        T.Resize(256),
        T.RandomCrop(224),
        T.RandomHorizontalFlip(),
        T.ToTensor(),
        T.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]) 


In [ ]:
import torch
model = torch.hub.load('mateuszbuda/brain-segmentation-pytorch', 'unet',
    in_channels=3, out_channels=1, init_features=32, pretrained=True)

import numpy as np
from PIL import Image
from torchvision import transforms
# print(type(train_dataset))
input_image = train_dataset.__getitem__(23)[0]

input_image = np.squeeze(input_image.numpy())
# print(type(input_image[0]))
m, s = np.mean(input_image[0], axis=(0, 1)), np.std(input_image[0], axis=(0, 1))
preprocess = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=m, std=s),
])
input_tensor = preprocess(input_image)
input_batch = input_tensor.unsqueeze(0)

if torch.cuda.is_available():
    input_batch = input_batch.to('cuda')
    model = model.to('cuda')

with torch.no_grad():
    output = model(input_batch)

print(torch.round(output[0]))

In [ ]:
input_image

(<PIL.Image.Image image mode=RGB size=2048x2048 at 0x7F003D7937D0>,
 {'boxes': [[1134, 765, 1322, 882],
   [1359, 1086, 1548, 1394],
   [768, 1044, 925, 1265]],
  'image_id': 'PE_074_121.jpg',
  'labels': [tensor(4), tensor(4), tensor(4)],
  'segmentation': [[[1167,
     765,
     1160,
     766,
     1153,
     766,
     1146,
     767,
     1139,
     770,
     1135,
     777,
     1135,
     784,
     1135,
     791,
     1135,
     798,
     1135,
     805,
     1135,
     812,
     1134,
     819,
     1134,
     826,
     1134,
     833,
     1134,
     840,
     1134,
     847,
     1134,
     854,
     1134,
     861,
     1135,
     868,
     1142,
     873,
     1149,
     874,
     1156,
     875,
     1163,
     876,
     1170,
     877,
     1177,
     877,
     1184,
     878,
     1191,
     878,
     1198,
     878,
     1205,
     879,
     1212,
     879,
     1219,
     880,
     1226,
     881,
     1233,
     881,
     1240,
     881,
     1247,
     882,
     1254